In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df_orig = pd.read_csv("cbecs2018_final_public.csv")
df_orig.head()

In [ ]:
display(df_orig.loc[(df_orig['PUBID'] > 0) & (df_orig['PBA'] == 2) & (df_orig['PUBCLIM'] == 4) & (df_orig['SQFT'] > 0), ['PUBID', 'PBA']])

In [ ]:
df_wk = df_orig.copy()
df_wk.head(2)

#### creates df_wk dataset

In [ ]:
df_wk = df_orig.iloc[:,[0,3,4,5,7,8,11,12,18,21,77,103,104,113,114,133,
    134,139,140,141,295,378,379,425,566,569,567,568,570,571,13]]
df_wk.head()

In [ ]:
# do not use
df_wk1 = df_orig.loc[(df_orig['PUBID'] > 0) &
    (df_orig['PBA'] == 2) &
    (df_orig['PUBCLIM'].isin([3, 4, 5])) &
    (df_orig['SQFT'] > 0) &
    (df_orig['WLCNS'] > 0) &
    (df_orig['RFCNS'] > 0) &
    (df_orig['BLDSHP'] > 0) &
    (df_orig['GLSSPC'] > 0) &
    (df_orig['YRCONC'] > 0) &
    (df_orig['ELHT1'] > 0) &
    (df_orig['NGHT1'] > 0) &
    (df_orig['NELVTR'] > 0) &
    (df_orig['WKHRSC'] > 0) &
    (df_orig['ELHT2'] > 0) &
    (df_orig['NGHT2'] > 0) &
    (df_orig['BOILER_EL'] > 0) &
    (df_orig['HTPMPH_EL'] > 0) &
    (df_orig['OTHTEQ_EL'] > 0) &
    (df_orig['PKGHT_NG'] > 0) &
    (df_orig['ELCOOL'] > 0) &
    (df_orig['ELWATR'] > 0) &
    (df_orig['NGWATR'] > 0) &
    (df_orig['NGOTH'] > 0) &
    (df_orig['BOILER_NG'] > 0) &
    (df_orig['ELBTU'] > 0) &
    (df_orig['NGBTU'] > 0)][
['PUBID', 'PBA', 'PUBCLIM', 'SQFT', 'WLCNS', 'RFCNS', 'BLDSHP', 'GLSSPC', 'YRCONC',
 'ELHT1', 'NGHT1', 'NELVTR', 'WKHRSC', 'ELHT2', 'NGHT2', 'BOILER_EL', 'HTPMPH_EL',
 'OTHTEQ_EL', 'PKGHT_NG', 'ELCOOL', 'ELWATR', 'NGWATR', 'NGOTH', 'BOILER_NG', 'ELBTU', 'NGBTU']]

df_wk1.head()





In [ ]:
df_wk.shape

In [ ]:
missing_values = df_wk.isna().sum()
print(missing_values)

In [ ]:
print(df_wk.info())

In [ ]:
print(df_wk.describe().round(0))

In [ ]:
print(df_wk[['PBA', 'WLCNS']].value_counts())

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 

# Make example dataframe 
df = pd.DataFrame([(1, 'Germany'), 
				(2, 'France'), 
				(3, 'Indonesia'), 
				(4, 'France'), 
				(5, 'France'), 
				(6, 'Germany'), 
				(7, 'UK'), 
				(8, 'India'), 
				(9, 'India'), 
				(10, 'Germany') 
				], 
				columns=['groupid', 'country'], 
				index=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']) 


# print country name and counts 
su3 = df['country'].value_counts() 
print(su3) 

# print 1st country count in a list 
su4 = df['country'].value_counts()[0] 
print(su4) 


In [ ]:
import pandas as pd
# Sample DataFrame
data = {'A': [1, 2, 3, 4, 5], 
        'B': [5, 4, 3, 2, 1], 
        'C': [1, 3, 5, 2, 4]}
df = pd.DataFrame(data)

# Calculate the correlation matrix
correlation_matrix = df.corr()

# Print the correlation matrix
print(correlation_matrix)